# Development notebook. No experiments or results here. For dev purposes




In [1]:
run init.ipynb

Done.


# Experiment on the best imputation technics in the case of single disribution in the bayesian framework 


First: multi-distribution - INDEPENDANT 

Second: Single-distribution - Which imputation method ? 

Third: NAM: Which config is best between encoding or imputation ?

Fourth: NAM: Which config is best between using indicator Variables or not ? 

Fifth: 

In [2]:
"""  Parameters of the experiment """
approach = 'multi_distributions'

range_dataset_name = ['blobs', 'circles', 'moons']
range_ratio_of_missing_values = [ 0.1, 0.2, 0.3, 0.]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.5, 0.25, 0.1]
range_missingness_pattern = [4, 5, 1, 3]

missing_data_handling = 'without'
imputation_method = 'without'
use_missing_indicator_variables = False

In [13]:
if False:
    df = create_df(folder_names=['experiments_24_june_duke', 'experiments_24_june', 'experiments_solidays', 'experiement_28_june', 'experiments_28_june_duke'])
else:
    df = pd.read_csv("/home01/sam/df_all_experiments.csv")

In [12]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = 20
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0



save=False

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:

        for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

            for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):
                
                
               
                if check_experiment_already_done(df, 
                          dataset_name=dataset_name, 
                          approach=approach,
                          imbalance_ratio=imbalance_ratio, 
                          imputation_method=imputation_method,
                          missingness_pattern=missingness_pattern, 
                          ratio_of_missing_values=ratio_of_missing_values, 
                          ratio_missing_per_class=ratio_missing_per_class):
                    print("Done")
                    continue
                    
                if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                    print("Not doing this.")
                    continue
                    
                else:
                    print("To be done")
                

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
Done
Done
Done
Done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be done
To be d

In [ ]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = 20
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0



save=False

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:

        for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

            for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):


                if  check_experiment_already_done(df, 
                                                  dataset_name=dataset_name, 
                                                  approach=approach,
                                                  imbalance_ratio=imbalance_ratio, 
                                                  imputation_method=imputation_method,
                                                  missingness_pattern=missingness_pattern, 
                                                  use_missing_indicator_variables=use_missing_indicator_variables,
                                                  ratio_of_missing_values=ratio_of_missing_values, 
                                                  ratio_missing_per_class=ratio_missing_per_class):


                    print("Experiment already done.")
                    continue
                    
                if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                    print("Not doing this.")

                print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                    missing_data_handling,
                                                                                                                                                      imputation_method, 
                                                                                                                                                      imbalance_ratio,
                                                                                                                                                      missingness_pattern,
                                                                                                                                                      ratio_of_missing_values, 
                                                                                                                                                      ratio_missing_per_class))
                
                start_time = time()
                
                

                dataset = DatasetGenerator(dataset_name=dataset_name, 
                                        num_samples=num_samples, 
                                        imbalance_ratio=imbalance_ratio, 
                                        missing_data_handling=missing_data_handling,
                                        imputation_method=imputation_method,
                                        verbosity=verbosity)


                # Creation of the missingness
                dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                     ratio_of_missing_values=ratio_of_missing_values, 
                                                     ratio_missing_per_class=ratio_missing_per_class)

                dataset.split_test_train()

                #X_all = dataset.X_test


                #print(np.argwhere( (X_all[:,0]>=2.5) | (X_all[:,0] <= -2.5) | (X_all[:,1]>=2.5) | (X_all[:,1] <= -2.5) ).squeeze())

                # Create the experiments
                exp = Experiments(dataset_name, 
                                  dataset=dataset, 
                                  purpose='classification', 
                                  approach = approach, 
                                  previous_experiment=None, 
                                  use_missing_indicator_variables=use_missing_indicator_variables,
                                  save_experiment=save, 
                                  verbosity=verbosity, 
                                  debug=False, 
                                  proportion_train=PROPORTION_TRAIN, 
                                  resolution=RESOLUTION, 
                                  bandwidth=BANDWIDTH,
                                  random_state=RANDOM_STATE)


                # Estimate the distributions 
                exp.fit()

                # Estimate the distributions 
                exp.predict()

                # Plot results
                #exp.plot()


                end_time = time()

                hours, rest = divmod(end_time - start_time, 3600)
                minutes, seconds = divmod(rest, 60)
                print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                #except:
                #    print("/!\. Error!\n")


blobs - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.1 ratio_per_class: None
